In [243]:
from platform import python_version

print(python_version())

from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem

from mendeleev import element

#import chemcoord as cc
from scipy import stats
from rdkit import Chem
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import glob
import csv
import ast
import os
import re
from os.path import exists
import rmsd
import pickle

#sns.set_context('paper', font_scale=1.1)

## CHANGE FOR YOUR COMPUTER ##
comp_loc = '/Users/z5380625/Documents/Research/4_BigData/'

3.10.6


# Get Unique Conformers List

In [219]:
def unique_conf_list(loc_num_atoms):
    log_files_confs = glob.glob(
        '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +
        '4_VibrationalCalcs/NotWorking_Conformers/*.log')
    o_files_confs = glob.glob(
        '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +
        '4_VibrationalCalcs/NotWorking_Conformers/*.o*')
    if len(log_files_confs) != 0:
        # Make lists for conformers
        log_confs_list = [(file.split('/')[-1].split('_harmonic')[0])
                               for file in log_files_confs]
        o_confs_list = [(file.split('/')[-1].split('.o')[0])
                               for file in o_files_confs]

        intersection_len  = len(list(set(log_confs_list) & set(o_confs_list)))
    #     if (intersection_len == len(log_confs_list)) & (intersection_len == len(o_confs_list)):
        filename_confs_list = [
            (file.split('/')[-1].split('_harmonic')[0].split('_CONF')[0],file.split('/')[-1].split('_harmonic')[0].split('_')[-1])
            for file in log_files_confs]        

        filename_confs_list = list(zip(*filename_confs_list))
        
        unique_confs_filenames, unique_confs_counts = np.unique(list(filename_confs_list[0]),return_counts=True)

        file_conf_comb_list = []
        for unique_mol, unique_count in zip(unique_confs_filenames,
                                            unique_confs_counts):
            conf_indices = np.where(
                np.asarray(filename_confs_list[0]) == unique_mol)[0]
            conf_list = []
            for ind in conf_indices:
                conf_num = np.asarray(filename_confs_list[1])[ind]
                conf_list.append(conf_num)
            if len(conf_list) == unique_count:
                file_conf_comb_list.append((unique_mol, conf_list))

        unique_list = file_conf_comb_list 
    else:
        unique_list = []

    return unique_list

In [220]:
unique_conf_list('6_Atoms/')

[('C2O2', ['CONF1']),
 ('C3O2', ['CONF1']),
 ('C3OS', ['CONF1']),
 ('C4N2', ['CONF1']),
 ('CN2O_1', ['CONF2']),
 ('CN4O', ['CONF1'])]

# Get Gibbs Free Energy

In [221]:
def int_get_gibbs_energy(location,loc_num_atoms,mol_id,energy_df):
    file = location+loc_num_atoms+'3_CREST_CENSO_Outputs/BWeights_GeomFiles_Conformers/'+mol_id+'_bweight_conformers.dat'
    if os.path.exists(file) == True:
        inp = open(file, 'r')
        
        energy_info = []
        for line in inp:
            line = line.strip()
            if line.startswith('CONF'):
                energy_info.append(line.split())
        for row in energy_info:
            if energy_info.index(row) > 0:
                energy_df.loc[len(energy_df)] = [row[0],float(row[5])*627.509*4.184,float(row[5])] # 4.184 is the conversion factor to kJ/mol
    return energy_df

In [222]:
def get_censo_gibbs_energy(location,loc_num_atoms,mol_id):
    energy_df = pd.DataFrame(columns = ['CONF#','G','RawG'])
    
    final_df = int_get_gibbs_energy(location,loc_num_atoms,mol_id,energy_df)
        
    final_df = final_df.sort_values('G')
    min_energy = np.min(final_df['G'].values)
    final_df['G'] = final_df['G'] - min_energy

    return final_df

# Get Error Type

In [244]:
comp_loc = '/Users/z5380625/Documents/Research/4_BigData/'
files_list = glob.glob(comp_loc+'?_Atoms')+glob.glob(comp_loc+'??_Atoms')
files_list.remove(comp_loc+'21_Atoms')
num_broke = 0

df = pd.DataFrame(columns = ['Formula','CONF#','Error Type'])
for file in files_list:
    loc_num_atoms = file.split('/')[-1]+'/'
    unique_mol_list = unique_conf_list(loc_num_atoms)
    for mol in unique_mol_list:
        mol_id = mol[0]
        mol_conf = mol[1]
        energy_df = get_censo_gibbs_energy(comp_loc,loc_num_atoms,mol_id)
        ground_state_conf = energy_df.loc[energy_df['G']==0.]['CONF#'].values[0]
        
        
        log_file = '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +'4_VibrationalCalcs/NotWorking_Conformers/'+mol_id+'_'+ground_state_conf+'_harmonic_conformers.log'
        o_file_confs = o_files_confs = glob.glob(
        '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +
        '4_VibrationalCalcs/NotWorking_Conformers/'+mol_id+'_'+ground_state_conf+'.o*')
        i=0
        n=0
        if ground_state_conf in mol_conf:
            with open(log_file, 'r') as f:
                for line in f.readlines():
                    if 'imaginary frequencies (negative Signs)' in line:
                        i+=1
                    elif 'Normal termination' in line:
                        n+=1
                if i > 0:
                    print('imaginary frequencies', mol_id,ground_state_conf)
                    error = 'ImagFreq'
                    num_broke += 1
                elif n != 2:
                    print('Termination Error',mol_id,ground_state_conf)
                    error = 'TermError'
                    num_broke += 1
            row_info = list([mol_id,ground_state_conf,error])
            df.loc[len(df)] = row_info
print(num_broke)
                        

Termination Error C2O2 CONF1
Termination Error C3O2 CONF1
Termination Error C3OS CONF1
Termination Error C4N2 CONF1
Termination Error CN2O_1 CONF2
Termination Error CN4O CONF1
Termination Error C2H3N3S_11 CONF1
imaginary frequencies C2H3N3S_7 CONF1
Termination Error C2H3N3S_8 CONF1
Termination Error C2H4NOP CONF1
Termination Error C2H4O2S_3 CONF1
imaginary frequencies C3H3NOS_7 CONF1
Termination Error C3H3NS2_4 CONF1
Termination Error C3H4OS_6 CONF1
Termination Error C4H3NO_2 CONF1
Termination Error C2H2N2S_1 CONF1
Termination Error C2H2N2S_2 CONF1
Termination Error C2H2OS2_1 CONF1
Termination Error C2H3NS2_2 CONF1
Termination Error C2H4OS_6 CONF1
Termination Error C2HN3O_1 CONF1
Termination Error C3H2N2O_2 CONF1
Termination Error C3H2OS2_2 CONF1
Termination Error C3H2S3_2 CONF1
Termination Error C3H4O_2 CONF1
Termination Error C4H2O CONF1
Termination Error C5HN_2 CONF1
Termination Error C6H2 CONF1
imaginary frequencies CH2N2O2_2 CONF1
Termination Error CH2N4S_2 CONF1
imaginary frequen

In [245]:
print(len(df[df['Error Type']=='TermError']))
print(len(df[df['Error Type']=='ImagFreq']))

120
9


In [246]:
print(129
/2914*100)

4.426904598490048


# Fixing Error with 17_Atoms

In [237]:
def unique_conf_list_output(loc_num_atoms):
    log_files_confs = glob.glob(
        '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +
        '4_VibrationalCalcs/OutputFiles/*.log')
    if len(log_files_confs) != 0:
        # Make lists for conformers
        log_confs_list = [(file.split('/')[-1].split('_harmonic')[0])
                               for file in log_files_confs]
        o_confs_list = [(file.split('/')[-1].split('.o')[0])
                               for file in o_files_confs]

        intersection_len  = len(list(set(log_confs_list) & set(o_confs_list)))
    #     if (intersection_len == len(log_confs_list)) & (intersection_len == len(o_confs_list)):
        filename_confs_list = [
            (file.split('/')[-1].split('_harmonic')[0].split('_CONF')[0],file.split('/')[-1].split('_harmonic')[0].split('_')[-1])
            for file in log_files_confs]        

    return filename_confs_list

In [238]:
def unique_conf_list_input(loc_num_atoms):
    pbs_files_confs = glob.glob(
        '/Users/z5380625/Documents/Research/4_BigData/' + loc_num_atoms +
        '4_VibrationalCalcs/InputFiles_Conformers/*.pbs')
    if len(pbs_files_confs) != 0:
        # Make lists for conformers
        pbs_confs_list = [(file.split('/')[-1].split('_harmonic')[0])
                               for file in pbs_files_confs]

        filename_confs_list = [
            (file.split('/')[-1].split('_harmonic')[0].split('_CONF')[0],file.split('/')[-1].split('_harmonic')[0].split('_')[-1])
            for file in pbs_files_confs]        

    return filename_confs_list

In [239]:
unique_output = unique_conf_list_output('17_Atoms/')
unique_input = unique_conf_list_input('17_Atoms/')


In [240]:
broken_list = list(set(unique_output) ^ set(unique_input))

## Make Vibrational Calculation Input Files 

In [241]:
# set folder location
comp_loc = '/Users/z5380625/Documents/Research/'
folder_loc = '17_Atoms'   

# set range for atoms
data = pd.read_csv(comp_loc+'4_BigData/'+folder_loc+'/biosig_CHNOPS.csv')
data = data.query("CHNOPS == 'Y'")
df_molecules = data.loc[(data['Tot_atoms'] == 17)]
df_molecules

,Unnamed: 0,Formula,Formula_ID,Tot_atoms,Tot_nH_atoms,Tot_H_atoms,SMILES,Dataset_no,Molecular_weight,IUPAC_chemname,...,EPISUITE,InChI_Code,InChI_Key,Boiling_Point,BP_basis,EPIsuite_output,MP,Basis,Life,CHNOPS
565,565,C5H12,C5H12_1,17,5,12,CC(CC)C,662,72.151,isopentane,...,CC(CC)C,"InChI=1S/C5H12/c1-4-5(2)3/h5H,4H2,1-3H3",InChIKey=QWTDNUCVQCZILF-UHFFFAOYSA-N,27.55462649,Ex,30.18(BP est) -119.04(MP est) 27.80(...,-159.9,Expt,Y,Y
566,566,C5H12,C5H12_2,17,5,12,CC(C)(C)C,663,72.151,neopentane,...,CC(C)(C)C,"InChI=1S/C5H12/c1-5(2,3)4/h1-4H3",InChIKey=CRSOQBOWXPBRES-UHFFFAOYSA-N,9.205,Ex,19.60(BP est) -113.42(MP est) 9.50(...,-16.4,Expt,N,Y
567,567,C5H12,C5H12_3,17,5,12,CCCCC,664,72.151,pentane,...,CCCCC,"InChI=1S/C5H12/c1-3-5-4-2/h3-5H2,1-2H3",InChIKey=OFBQJSOFQDEBGM-UHFFFAOYSA-N,36.02657459,Ex,46.01(BP est) -106.92(MP est) 36.00(...,-129.7,Expt,Y,Y
1644,1644,C5H11N,C5H11N_1,17,6,11,NCC1(CC1)C,1850,85.150,(1-methylcyclopropyl)methanamine,...,NCC1(CC1)C,"InChI=1S/C5H11N/c1-5(4-6)2-3-5/h2-4,6H2,1H3",InChIKey=NIJMHZCHTZXLJX-UHFFFAOYSA-N,95.55,Ex,103.22(BP est) -27.67(MP est) ----(...,-27.67,Pred,N,Y
1645,1645,C5H11N,C5H11N_2,17,6,11,NCC1C(C)C1,1851,85.150,(2-methylcyclopropyl)methanamine,...,NCC1C(C)C1,"InChI=1S/C5H11N/c1-4-2-5(4)3-6/h4-5H,2-3,6H2,1H3",InChIKey=BRWRRWXQMCPPRZ-UHFFFAOYSA-N,108.88,Pr,108.88(BP est) -40.09(MP est) ----(...,-40.09,Pred,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12168,12168,C5H11N,C5H11N_46,17,6,11,C/C=C(C)/CN,14866,85.150,2-methylbut-2-en-1-amine,...,C/C=C(C)/CN,"InChI=1S/C5H11N/c1-3-5(2)4-6/h3H,4,6H2,1-2H3",InChIKey=PDLOGTOXFRBJKR-UHFFFAOYSA-N,108,Ex,112.54(BP est) -55.39(MP est) ----(...,-55.39,Pred,N,Y
12184,12184,C5H11N,C5H11N_47,17,6,11,C/C=C/N(C)C,14882,85.150,"N,N-dimethylprop-1-en-1-amine",...,C/C=C/N(C)C,"InChI=1S/C5H11N/c1-4-5-6(2)3/h4-5H,1-3H3",InChIKey=NXBBFAKHXAMPOM-UHFFFAOYSA-N,82,Ex,78.52(BP est) -83.74(MP est) ----(...,-83.74,Pred,N,Y
12185,12185,C5H11N,C5H11N_48,17,6,11,C/C=C/CNC,14883,85.150,N-methylbut-2-en-1-amine,...,C/C=C/CNC,"InChI=1S/C5H11N/c1-3-4-5-6-2/h3-4,6H,5H2,1-2H3",InChIKey=UUBPFQGETKBNFO-UHFFFAOYSA-N,82,Ex,100.43(BP est) -67.24(MP est) ----(...,-67.24,Pred,N,Y
12186,12186,C5H11N,C5H11N_49,17,6,11,CC(N=C)(C)C,14884,85.150,N-tert-butylmethanimine,...,CC(N=C)(C)C,"InChI=1S/C5H11N/c1-5(2,3)6-4/h4H2,1-3H3",InChIKey=RSUCYDXEFFBUSN-UHFFFAOYSA-N,66.5,Ex,47.70(BP est) -104.83(MP est) ----(...,-104.83,Pred,N,Y


In [242]:
## Change Conformers/Rotamer Extensions ##

extension = ['Conformers','conformers','CONF'] #

# ----------------------------------------------------------
# Input File Generation for Gaussian Vibrational Calculation
# ----------------------------------------------------------

from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)


files_names = []
for mol in broken_list:
    files_names.append('/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/'+mol[0]+'_'+mol[1]+'.xyz')

print(files_names)

two_mult = ['CH','HCO','NO2','OH','SH','BO','ClO','CN','HO2']


molecules_considered = list(df_molecules['Formula_ID'])

for file in files_names:

    filen = file.split('/')[-1].split('.xyz')[0].split('_'+extension[2])[0]
    filen = filen.split('p')[0]
    filename = file.split('/')[-1].split('.xyz')[0]
    if filen in molecules_considered:
        try:
            # Copies the template file, changing its name to the one for the molecule being considered.
            # https://thispointer.com/python-how-to-copy-files-from-one-location-to-another-using-shutil-copy/
            shutil.copy(comp_loc+'4_BigData/'+folder_loc+'/Templates/harm_gaussian.gjf', comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_Broken/'+filename+'_harmonic_'+extension[1]+'.gjf')
            
            basis_data = []
            elements_list = []
            atomic_numbers = []
            basis_str = ''

            # These for loops store the element composition in the molecular formula including the atomic numbers.
            for letter in re.findall('[A-Z][^A-Z]*', filen.rstrip().split('_'+extension[2])[0]):
                symbol = re.split(r'[0-9]', letter.split()[0])[0]
                poss_under = re.split(r'(_)', symbol.split()[0])[0]
                elements_list.append(poss_under)

            for atom in elements_list:
                        symbol = element(atom)
                        atomic_numbers.append(symbol.atomic_number)

            atomic_numbers.sort()

            # With the atomic number from the molecular formula, this part of the code stores the corresponding elements
#             from the user-defined basis set.
            basis_file = open(comp_loc+'4_BigData/'+folder_loc+'/UserBasisSet/Def2TZVPD.txt',"r")
            contents = basis_file.read()
            basis_dict = ast.literal_eval(contents)
            for number in atomic_numbers:
                if number in basis_dict:
                    basis_data.append(basis_dict[number])

            basis_str = '\n'.join(basis_data)

#             # Saves the info for the respective geometry into the geom_data list.
            with open(file,'r') as inp_geom:
                geom_data = inp_geom.read()
                geom_data = geom_data.split('\n',2)[-1]
                inp_geom.close()
                
            # Replaces the selected keywords (i.e. {molecule}, {geometry} and {basis}) with the
            # corresponding values.
            # https://www.kite.com/python/answers/how-to-update-and-replace-text-in-a-file-in-python
            with open(comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_Broken/'+filename+'_harmonic_'+extension[1]+'.gjf','r+') as inp_template:
                template_data = inp_template.read()
                template_data = re.sub('{cpu}', '8', template_data)
                template_data = re.sub('{mem}', '32', template_data)
                template_data = re.sub('{geometry}', geom_data, template_data)
                template_data = re.sub('{molecule}', filename.rstrip(), template_data)
                template_data = re.sub('{charge}', '0', template_data)
                template_data = re.sub('{basis_data}', basis_str, template_data)
                
                # Considering different values of multiplicity
                if filen in two_mult:
                    template_data = re.sub('{multiplicity}', '2', template_data)
                else:
                    template_data = re.sub('{multiplicity}', '1', template_data)
            
                
                inp_template.seek(0)
                inp_template.write(template_data)
                inp_template.truncate()
                
            # Copies submission file template and pastes the appropriate input file name.
            shutil.copy(comp_loc+'4_BigData/'+folder_loc+'/Templates/harm_gaussian_submission.pbs', comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_Broken/'+filename+'_harmonic_'+extension[1]+'.pbs')
            with open(comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_Broken/'+filename+'_harmonic_'+extension[1]+'.pbs','r+') as inp_sub:
                submission_data = inp_sub.read()
                submission_data = re.sub('{molecule}', filename.rstrip(), submission_data)
                submission_data = re.sub('{extension}', extension[1].rstrip(), submission_data)
                submission_data = re.sub('{time}', '04:00:00', submission_data)
                submission_data = re.sub('{mem}', '32', submission_data)
                submission_data = re.sub('{cpus}', '8', submission_data)
                submission_data = re.sub('{extension}', extension[1], submission_data)

                inp_sub.seek(0)
                inp_sub.write(submission_data)
                inp_sub.truncate()
                
        except:
            
            print(filen)
        

['/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C4H11NO_5_CONF26.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C5H11N_21_CONF3.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C5H11N_37_CONF2.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C4H11NO_21_CONF22.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C5H11N_46_CONF1.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C4H11OP_2_CONF4.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C5H11N_21_CONF4.xyz', '/Users/z5380625/Documents/Research/4_BigData/17_Atoms/3_CREST_CENSO_Outputs/FinalGeoms_Conformers/C4H11OP_4_CONF12.xyz', '/Users/z5380625/Documents/